# PII W and B Training
-removed stride compared to reference
-raw data is full competition training comes from base_data artifact

In [1]:
# set up in config
#TODO check reference dataframe and raw_data
#re

# Imports

In [2]:
!pip install seqeval evaluate transformers -q
!pip download  seqeval -q #for use offline in inference notebook

In [3]:
!pip install --upgrade wandb -q
import wandb

In [4]:
from pathlib import Path
import os

import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features
import numpy as np
import pandas as pd

2024-04-10 04:36:46.546131: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 04:36:46.546237: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 04:36:46.805556: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Util Functions

In [5]:
# https://www.kaggle.com/code/thedrcat/pii-data-detection-train-with-w-b/input?select=utils.py
# https://www.kaggle.com/code/valentinwerner/915-deberta3base-inference?scriptVersionId=161126788
# https://www.kaggle.com/code/sinchir0/visualization-code-using-displacy

import os
import json
import numpy as np
import pandas as pd
import wandb
from concurrent.futures import ProcessPoolExecutor, as_completed
from tqdm.auto import tqdm
from spacy.tokens import Span
from spacy import displacy
import argparse
from ast import literal_eval
from transformers import Trainer
from torch.nn import CrossEntropyLoss
from scipy.special import softmax

def str2bool(v):
    "Fix Argparse to process bools"
    if isinstance(v, bool):
        return v
    if v.lower() == 'true':
        return True
    elif v.lower() == 'false':
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

def parse_args(config):
    print("Running with the following config")
    parser = argparse.ArgumentParser(description='Run training baseline')
    for k,v in config.__dict__.items():
        parser.add_argument('--'+k, type=type(v) if type(v) is not bool else str2bool, 
                            default=v, 
                            help=f"Default: {v}")
    args = vars(parser.parse_args())
    
    # update config with parsed args
    for k, v in args.items():
        try:
            # attempt to eval it it (e.g. if bool, number, or etc)
            attempt = literal_eval(v)
        except (SyntaxError, ValueError):
            # if that goes wrong, just use the string
            attempt = v
        setattr(config, k, attempt)
        print(f"--{k}:{v}")

#Mine
def threshold_preds(predictions, ds, threshold = 0.9) -> pd.DataFrame:
    # Scale last dimension to probabilities for interpretability
    pred_softmax = softmax(predictions, axis=-1)

    preds = predictions.argmax(-1)
    preds_without_O = pred_softmax[:,:,:12].argmax(-1)
    O_preds = pred_softmax[:,:,12]
    
    #Choose label with max probability
    preds_final = predictions.argmax(-1)
    # or if using threshold
    #preds_final = np.where(O_preds < threshold, preds_without_O , preds)

    return preds_final

#My edits
def preds_to_df (preds_final, ds, id2label):
    """
    Create a DataFrame of parsed predictions.

    Parameters:
    - preds_final (list): List of predictions.
    - ds (Dataset): Dataset containing the token maps, offset mappings, tokens, and documents.
    - id2label (dict): Dictionary mapping label IDs to labels.

    Returns:
    - DataFrame: DataFrame containing the submission information.
    """
    #TODO based on next 
    # Create lists of submission information
    triplets = set() #improve runtime as recommended by trip
    row, document, token, label, token_str = [], [], [], []

    for i, (p, token_map, offsets, tokens, doc, indices) in enumerate(zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"], ds["token_indices"])):
        # Iterate through each prediction and its offset
        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            label_pred = id2label[str(token_pred)]  # Predicted label #CHECK remove str

            if start_idx + end_idx == 0: continue   # For special token or padding token

            if token_map[start_idx] == -1:  # Label is for whitespace so go to next
                start_idx += 1

            # Ignore leading whitespace token "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                start_idx += 1
            
            # Break if index exceeds the length of token mapping
            if start_idx >= len(token_map): break
            
            token_id = token_map[start_idx]  # Token ID at start of index
            #CHECK original token id= token_map[] vs indices[original token id]

            # Ignore "O" predictions and whitespace predictions
            if label_pred != "O" and token_id != -1:
                triplet = (label_pred, token_id, tokens[token_id])

                # Add triplet if not in list of triplets
                if triplet not in triplets:
                    row.append(i)
                    document.append(doc)
                    token.append(token_id)
                    label.append(label_pred)
                    token_str.append(tokens[token_id])
                    triplets.add(triplet)

    # Create a DataFrame of submission information
    df = pd.DataFrame({
        "eval_row": row, #CHECK is this needed or only for stride
        "document": document,
        "token": token,
        "label": label,
        "token_str": token_str
    })

    #TODO try out
    df = df.drop_duplicates().reset_index(drop=True)
    df["row_id"] = list(range(len(df)))

    return df

def parse_predictions(predictions, id2label, ds, threshold=0.9):
    
    pred_softmax = np.exp(predictions) / np.sum(np.exp(predictions), axis = 2).reshape(predictions.shape[0],predictions.shape[1],1)
    preds = predictions.argmax(-1)
    preds_without_O = pred_softmax[:,:,:12].argmax(-1)
    O_preds = pred_softmax[:,:,12]
    preds_final = np.where(O_preds < threshold, preds_without_O , preds)

    triplets = set()
    row, document, token, label, token_str = [], [], [], [], []
    for i, (p, token_map, offsets, tokens, doc, indices) in enumerate(zip(preds_final, ds["token_map"], ds["offset_mapping"], ds["tokens"], ds["document"], ds["token_indices"])):

        for token_pred, (start_idx, end_idx) in zip(p, offsets):
            label_pred = id2label[token_pred]

            if start_idx + end_idx == 0: continue

            if token_map[start_idx] == -1:
                start_idx += 1

            # ignore "\n\n"
            while start_idx < len(token_map) and tokens[token_map[start_idx]].isspace():
                start_idx += 1

            if start_idx >= len(token_map): break

            #CHECK
            token_id = token_map[start_idx] #token ID at the start of the index
#             original_token_id = token_map[start_idx]
#             token_id = indices[original_token_id]

            # ignore "O" predictions and whitespace preds
            if label_pred != "O" and token_id != -1:
                triplet = (label_pred, token_id, tokens[token_id])

                if triplet not in triplets:
                    row.append(i)
                    document.append(doc)
                    token.append(token_id)
                    label.append(label_pred)
                    token_str.append(tokens[token_id])
                    triplets.add(triplet)

    df = pd.DataFrame({
        "eval_row": row,
        "document": document,
        "token": token,
        "label": label,
        "token_str": token_str
    })

    df = df.drop_duplicates().reset_index(drop=True)

    df["row_id"] = list(range(len(df)))
    return df


# Mapping of named colors to their RGB equivalents
named_color_to_rgb = {
    "aqua": (0, 255, 255),
    "skyblue": (135, 206, 235),
    "limegreen": (50, 205, 50),
    "lime": (0, 255, 0),
    "hotpink": (255, 105, 180),
    "lightpink": (255, 182, 193),
    "purple": (128, 0, 128),
    "rebeccapurple": (102, 51, 153),
    "red": (255, 0, 0),
    "salmon": (250, 128, 114),
    "silver": (192, 192, 192),
    "lightgray": (211, 211, 211),
    "brown": (165, 42, 42),
    "chocolate": (210, 105, 30),
    # Add the rest of your named colors and their RGB values here
}

def get_rgba(color_name, opacity):
    """Convert a named color and opacity to an rgba string."""
    rgb = named_color_to_rgb[color_name]  # Get the RGB values for the named color
    return f'rgba({rgb[0]}, {rgb[1]}, {rgb[2]}, {opacity})'


def visualize_prediction_html(tokenizer, row_id, predictions, id2label, ds, threshold=0.7):
    # Define colors for each label using the provided color map
    label_colors = {
        "B-NAME_STUDENT": "aqua",
        "I-NAME_STUDENT": "skyblue",
        "B-EMAIL": "limegreen",
        "I-EMAIL": "lime",
        "B-USERNAME": "hotpink",
        "I-USERNAME": "lightpink",
        "B-ID_NUM": "purple",
        "I-ID_NUM": "rebeccapurple",
        "B-PHONE_NUM": "red",
        "I-PHONE_NUM": "salmon",
        "B-URL_PERSONAL": "silver",
        "I-URL_PERSONAL": "lightgray",
        "B-STREET_ADDRESS": "brown",
        "I-STREET_ADDRESS": "chocolate",
    }

    # Process predictions to get softmax probabilities, excluding the last column ("O" class)
    pred_softmax = np.exp(predictions[:, :, :-1]) / np.sum(np.exp(predictions[:, :, :-1]), axis=2, keepdims=True)
    
    # Find the document in the dataset using row_id
    doc_tokens = [tokenizer.decode(x) for x in ds["input_ids"][row_id]]
    doc_predictions = pred_softmax[row_id]
    doc_labels = doc_predictions.argmax(-1)

    # Start building HTML content
    html_content = '<div style="font-family: Arial; color: black; font-size: larger;">'
    for token, token_pred, token_softmax in zip(doc_tokens, doc_labels, doc_predictions):
        label_pred = id2label[token_pred]
        if label_pred != "O":
            color = label_colors.get(label_pred, "#FFFFFF")  # Default color is white if label not found
            opacity = np.max(token_softmax)  # Use the max softmax score as opacity
            rgba_color = get_rgba(color, opacity)
            html_content += f'<span style="background-color: {rgba_color};">{token}</span> '
        else:
            html_content += f'{token} '

    # Add all tokens with probability above threshold
    high_prob_indices = np.where(doc_predictions.max(-1) > threshold)
    for index in high_prob_indices[0]:
        high_prob_label = id2label[doc_labels[index]]
        high_prob_value = doc_predictions.max(-1)[index]
        high_prob_color = label_colors.get(high_prob_label, "#FFFFFF")
        rgba_color = get_rgba(high_prob_color, high_prob_value)
        html_content += f'<p>High probability token: <span style="background-color: {rgba_color};">{doc_tokens[index]}</span> Label: {high_prob_label} Probability: {high_prob_value}</p>'
    
    # Add legend for class-color mapping
    html_content += '<div><p>Legend:</p><ul>'
    for label, color in label_colors.items():
        html_content += f'<li><span style="background-color: {get_rgba(color, 1)};">{label}</span></li>'
    html_content += '</ul></div></div>'
    return html_content

#CHECK- modified from https://www.kaggle.com/code/thedrcat/pii-data-detection-train-with-w-b/input
def get_reference_df(artifact, filename='val_data.parquet'): 
    raw_artifact = wandb.use_artifact(artifact)
    raw_artifact_dir = raw_artifact.download()
    raw_df = pd.read_parquet(raw_artifact_dir + f'/{filename}')
    
    ref_df = raw_df[['document', 'tokens', 'labels']].copy()
    ref_df = ref_df.explode(['tokens', 'labels']).reset_index(drop=True).rename(columns={'tokens': 'token', 'labels': 'label'})
    ref_df['token'] = ref_df.groupby('document').cumcount()
        
    reference_df = ref_df[ref_df['label'] != 'O'].copy()
    reference_df = reference_df.reset_index().rename(columns={'index': 'row_id'})
    reference_df = reference_df[['row_id', 'document', 'token', 'label']].copy()
    
    return reference_df

#TODO maybe - optimize visualize functions
# def process_html(tokenizer, index, predictions, id2label, valid_ds, threshold=0.9):
#     return index, wandb.Html(visualize_prediction_html(tokenizer, index, predictions, id2label, valid_ds, threshold=0.9))

# def generate_htmls_concurrently(viz_df, tokenizer, predictions, id2label, valid_ds, threshold=0.9):
#     results_with_index = []
#     with ProcessPoolExecutor() as executor:
#         futures = [executor.submit(process_html, tokenizer, i, predictions, id2label, valid_ds, threshold=0.9) for i in viz_df.index.values.tolist()]
#         for future in tqdm(as_completed(futures), total=len(viz_df)):
#             results_with_index.append(future.result())
    
#     # Sort the results by index to maintain the original order
#     results_with_index.sort(key=lambda x: x[0])
#     htmls = [result[1] for result in results_with_index]
#     return htmls

# def visualize(row, nlp):
#     options = {
#         "colors": {
#             "B-NAME_STUDENT": "aqua",
#             "I-NAME_STUDENT": "skyblue",
#             "B-EMAIL": "limegreen",
#             "I-EMAIL": "lime",
#             "B-USERNAME": "hotpink",
#             "I-USERNAME": "lightpink",
#             "B-ID_NUM": "purple",
#             "I-ID_NUM": "rebeccapurple",
#             "B-PHONE_NUM": "red",
#             "I-PHONE_NUM": "salmon",
#             "B-URL_PERSONAL": "silver",
#             "I-URL_PERSONAL": "lightgray",
#             "B-STREET_ADDRESS": "brown",
#             "I-STREET_ADDRESS": "chocolate",
#         }
#     }
#     doc = nlp(row.full_text)
#     doc.ents = [
#         Span(doc, idx, idx + 1, label=label)
#         for idx, label in enumerate(row.labels)
#         if label != "O"
#     ]
#     html = displacy.render(doc, style="ent", jupyter=False, options=options)
#     return html

# def convert_for_upload(viz_df):
#     mapping = {'index': 'str',
#      'document_x': 'str',
#      'tokens': 'str',
#      'trailing_whitespace': 'str',
#      'labels': 'str',
#      'token_indices': 'str',
#      'full_text': 'str',
#      'unique_labels': 'str',
#      'EMAIL': 'str',
#      'ID_NUM': 'str',
#      'NAME_STUDENT': 'str',
#      'PHONE_NUM': 'str',
#      'STREET_ADDRESS': 'str',
#      'URL_PERSONAL': 'str',
#      'USERNAME': 'str',
#      'OTHER': 'str',
#      'document_y': 'str',
#      'token': 'str',
#      'label': 'str',
#      'token_str': 'str',
#     }
#     for key,type in mapping.items():
#         viz_df[key] = viz_df[key].astype(type)
#     return viz_df

def filter_errors(eval_df, preds_df):
    target_strings = []
    for i,row in eval_df.iterrows():
        target_string = [f'{t}: {l}' for t,l in zip(row.tokens, row.labels) if l != "O"]
        target_strings.append(' '.join(target_string))
    
    pred_strings = []
    for i in range(len(eval_df)):
        i_preds = preds_df[preds_df.eval_row == i]
        if len(i_preds) > 0:
            pred_string = [f'{t}: {l}' for t,l in zip(i_preds.token_str, i_preds.label)]
        else: 
            pred_string = []
        pred_strings.append(' '.join(pred_string))
    
    eval_df['target_string'] = target_strings
    eval_df['pred_string'] = pred_strings
    eval_df['error'] = eval_df['target_string'] != eval_df['pred_string']
    return eval_df[eval_df.error == True]

class CustomTrainer(Trainer):
    def __init__(self, *args, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        # Assuming class_weights is a Tensor of weights for each class
        self.class_weights = class_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        # Extract labels
        labels = inputs.pop("labels")
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        # Reshape for loss calculation
        loss_fct = CrossEntropyLoss(weight=self.class_weights)
        if self.label_smoother is not None and "labels" in inputs:
            loss = self.label_smoother(outputs, inputs)
        else:
            loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        
        return (loss, outputs) if return_outputs else loss
    
# https://www.kaggle.com/competitions/llm-detect-ai-generated-text/discussion/465970#2589607
#CHECK
def upload_kaggle_dataset(storage_dir, dataset_name, owner="thedrcat"):
    """
    :param storage_dir: upload storage dir to kaggle as dataset
    :param dataset_name: name of the dataset
    :param owner: name of the dataset owner
    """
    print(f"creating metadata...")
    os.system(f"kaggle datasets init -p {storage_dir}")

    print(f"updating metadata...")
    with open(os.path.join(storage_dir, "dataset-metadata.json"), "r") as f:
        metadata = json.load(f)

    metadata['title'] = dataset_name
    metadata['id'] = f"{owner}/{dataset_name}".replace("_", "-")

    print(f"saving updated metadata...")
    with open(os.path.join(storage_dir, "dataset-metadata.json"), "w") as f:
        json.dump(metadata, f)

    print("uploading the dataset ...")
    os.system(f"kaggle datasets create -p {storage_dir}")
    print("done!")

# data Functions

In [6]:
import numpy as np
from datasets import Dataset

#prep data for NER training by tokenize the text and align labels to tokens
def tokenize(example, tokenizer, label2id, max_length):
    """This function ensures that the text is correctly tokenized and the labels 
    are correctly aligned with the tokens for NER training.

    Args:
        example (dict): The example containing the text and labels.
        tokenizer (Tokenizer): The tokenizer used to tokenize the text.
        label2id (dict): A dictionary mapping labels to their corresponding ids.
        max_length (int): The maximum length of the tokenized text.

    Returns:
        dict: The tokenized example with aligned labels.

    Reference: credit to https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
    """

    # rebuild text from tokens
    text = []
    labels = []
    token_map = [] 
    
    idx = 0

    #iterate through tokens, labels, and trailing whitespace using zip to create tuple from three lists
    for t, l, ws in zip(
        example["tokens"], example["provided_labels"], example["trailing_whitespace"]
    ):
        
        text.append(t)
        token_map.extend([idx]*len(t)) 
        #extend so we can add multiple elements to end of list if ws
        labels.extend([l] * len(t))
        
        if ws:
            text.append(" ")
            labels.append("O")
            token_map.append(-1) #CHECK
            
        idx += 1

    #Tokenize text and return offsets for start and end character position. Limit length of tokenized text.
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length, truncation=True) #TODO check truncation

    #convert to np array for indexing
    labels = np.array(labels)

    # join text list into a single string 
    text = "".join(text)
    token_labels = []

    #iterate through each tolken
    for start_idx, end_idx in tokenized.offset_mapping:
        #if special tolken (CLS token) then append O
        #CLS : classification token added to the start of each sequence
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        #append orginal label to token_labels
        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length,"token_map": token_map, } 

#helper function for loading to dataset
def load_process_ds_helper(json_path, tokenizer, max_length, label2id):
    # Load data from JSON file into a dict
    data = json.load(open(json_path))
    
    # Convert data to Hugging Face Dataset object
    ds = Dataset.from_dict({
        "full_text": [x["full_text"] for x in data],
        "document": [str(x["document"]) for x in data],
        "tokens": [x["tokens"] for x in data],
        "trailing_whitespace": [x["trailing_whitespace"] for x in data],
        "provided_labels": [x["labels"] for x in data],
        "token_indices": [x["token_indices"] for x in data],
    })

    # Map the tokenize function to the Dataset
    ds = ds.map(
        tokenize,
        fn_kwargs={      # pass keyword args
            "tokenizer": tokenizer,
            "label2id": label2id,
            "max_length": max_length
        }, 
        num_proc=3
    )

    return ds

#create dataset if using wandb
def create_dataset(data, tokenizer, max_length, label2id):
    '''
    data(pandas.DataFrame): for wandb artifact
    '''
    # Convert data to Hugging Face Dataset object
    ds = Dataset.from_dict({
        "full_text": data.full_text.tolist(),
        "document": data.document.tolist(),
        "tokens": data.tokens.tolist(),
        "trailing_whitespace": data.trailing_whitespace.tolist(),
        "provided_labels": data.labels.tolist(),
        "token_indices": data.token_indices.tolist(),
    })

    # Map the tokenize function to the Dataset
    ds = ds.map(
        tokenize,
        fn_kwargs={      # pass keyword args
            "tokenizer": tokenizer,
            "label2id": label2id,
            "max_length": max_length
        }, 
        num_proc=2
    )

    return ds

# Metric Functions

In [7]:
# https://www.kaggle.com/code/conjuring92/pii-metric-fine-grained-eval

from collections import defaultdict
from typing import Dict
# from utils import parse_predictions #SCRIPT version

class PRFScore:
    """A precision / recall / F score."""

    def __init__(
        self,
        *,
        tp: int = 0,
        fp: int = 0,
        fn: int = 0,
    ) -> None:
        self.tp = tp
        self.fp = fp
        self.fn = fn

    def __len__(self) -> int:
        return self.tp + self.fp + self.fn

    def __iadd__(self, other):  # in-place add
        self.tp += other.tp
        self.fp += other.fp
        self.fn += other.fn
        return self

    def __add__(self, other):
        return PRFScore(
            tp=self.tp + other.tp, fp=self.fp + other.fp, fn=self.fn + other.fn
        )

    def score_set(self, cand: set, gold: set) -> None:
        self.tp += len(cand.intersection(gold))
        self.fp += len(cand - gold)
        self.fn += len(gold - cand)

    @property
    def precision(self) -> float:
        return self.tp / (self.tp + self.fp + 1e-100)

    @property
    def recall(self) -> float:
        return self.tp / (self.tp + self.fn + 1e-100)

    @property
    def f1(self) -> float:
        p = self.precision
        r = self.recall
        return 2 * ((p * r) / (p + r + 1e-100))

    @property
    def f5(self) -> float:
        beta = 5
        p = self.precision
        r = self.recall

        fbeta = (1+(beta**2))*p*r / ((beta**2)*p + r + 1e-100)
        return fbeta

    def to_dict(self) -> Dict[str, float]:
        return {"p": self.precision, "r": self.recall, "f5": self.f5}


def compute_metrics(p, id2label, valid_ds, valid_df, threshold=0.9):
    """
    Compute the LB metric (lb) and other auxiliary metrics
    """
    predictions, labels = p
    
    pred_df = parse_predictions(predictions, id2label, valid_ds, threshold=threshold)
    
    references = {(row.document, row.token, row.label) for row in valid_df.itertuples()}
    predictions = {(row.document, row.token, row.label) for row in pred_df.itertuples()}
    
    score_per_type = defaultdict(PRFScore)
    references = set(references)

    for ex in predictions:
        pred_type = ex[-1] # (document, token, label)
        if pred_type != 'O':
            pred_type = pred_type[2:] # avoid B- and I- prefix
            
        if pred_type not in score_per_type:
            score_per_type[pred_type] = PRFScore()

        if ex in references:
            score_per_type[pred_type].tp += 1
            references.remove(ex)
        else:
            score_per_type[pred_type].fp += 1

    for doc, tok, ref_type in references:
        if ref_type != 'O':
            ref_type = ref_type[2:] # avoid B- and I- prefix
        
        if ref_type not in score_per_type:
            score_per_type[ref_type] = PRFScore()
        score_per_type[ref_type].fn += 1

    totals = PRFScore()
    
    for prf in score_per_type.values():
        totals += prf

    results = {
        "ents_p": totals.precision,
        "ents_r": totals.recall,
        "ents_f5": totals.f5,
        "ents_per_type": {k: v.to_dict() for k, v in score_per_type.items() if k!= 'O'},
    }
    
    # Unpack nested dictionaries
    final_results = {}
    for key, value in results.items():
        if isinstance(value, dict):
            for n, v in value.items():
                if isinstance(v, dict):
                    for n2, v2 in v.items():
                        final_results[f"{key}_{n}_{n2}"] = v2
                else:
                    final_results[f"{key}_{n}"] = v              
        else:
            final_results[key] = value
            
    return final_results

In [8]:
# #CHECK Alt compute metrics that i have editted

# #def compute_metrics(p, id2label, valid_ds, valid_df, threshold=0.9):
# from seqeval.metrics import recall_score, precision_score
# from seqeval.metrics import classification_report
# from seqeval.metrics import f1_score

# def compute_metrics(p, all_labels):
#     """Compute the F1, recall, precision metrics for a NER task.

#     Args:
#         p (Tuple[np.ndarray, np.ndarray]): The predictions and labels.
#         all_labels (List[str]): The list of all possible labels.

#     Returns:
#         Dict[str, float]: The computed metrics (recall, precision, f1_score).
#     Ref: https://www.kaggle.com/code/valentinwerner/915-deberta3base-training/notebook
#     """
#     #Note: seqeval framework for sequence labeling like NER
    
#     # Unpack the predictions and labels
#     predictions, labels = p
#     predictions = np.argmax(predictions, axis=2)

#     # Remove ignored index (special tokens)
#     true_predictions = [
#         [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
#     true_labels = [
#         [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
#         for prediction, label in zip(predictions, labels)
#     ]
    
#     recall = recall_score(true_labels, true_predictions)
#     precision = precision_score(true_labels, true_predictions)
#     f5_score = (1 + 5*5) * recall * precision / (5*5*precision + recall + 1e-10)
    
#     results = {
#         'recall': recall,
#         'precision': precision,
#         'f5': f5_score
#     }
#     return results

# Training Script


In [9]:
import os
from itertools import chain
from functools import partial
from transformers import AutoTokenizer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import pandas as pd
from types import SimpleNamespace
import torch
import wandb
import spacy

# Import necessary functions and classes from other files
# from metric import compute_metrics
# from data import create_dataset
# from utils import get_reference_df, parse_predictions
# from utils import filter_errors, generate_htmls_concurrently, visualize, convert_for_upload
# from utils import CustomTrainer
# from utils import upload_kaggle_dataset, parse_args

# Define the project name for Weights & Biases
WANDB_PROJECT = 'pii'

#Define other names
TRAIN_ARTIFACT_NAME = 'mini_no_overlap'
VAL_ARTIFACT_NAME ='val_data'


# Define the configuration for the experiment #TODO modify later
config = SimpleNamespace(
    experiment='pii000',
    threshold=0.9,
    o_weight=0.05,
    raw_artifact='csci566sp24/pii/base_data:v0',
    train_artifact = 'csci566sp24/pii/mini_no_overlap_data:v0',
    val_artifact = 'csci566sp24/pii/val_data:v0',
    external_data_1='none',
    external_data_2='none',
    external_data_3='none',
    external_data_4='none',
    external_data_5='none',
    output_dir="output",
    inference_max_length=1024, #input sequence for inference
    training_max_length=1024,  #inference sequence for training
    training_model_path="microsoft/deberta-v3-xsmall", #base model
    fp16=True,
    learning_rate=1e-5,
    num_train_epochs=0.1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    report_to="wandb",
    evaluation_strategy="epoch",
    do_eval=True,
    save_total_limit=1,
    logging_steps=10,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    weight_decay=0.01,
    
#     load_best_model_at_end = True,
#     metric_for_best_model = "f1",
#     greater_is_better = True,
)

def main(config):
    # Initialize Weights & Biases
    wandb.init(project=WANDB_PROJECT, job_type='train', config=config)
    config = wandb.config
    
   # Load the training data
    train_artifact = wandb.use_artifact(config.train_artifact)
    train_artifact_dir = train_artifact.download()
    train_df = pd.read_parquet(train_artifact_dir + '/'+ TRAIN_ARTIFACT_NAME +'.parquet')

    # Load the validation data
    val_artifact = wandb.use_artifact(config.val_artifact)
    val_artifact_dir = val_artifact.download()
    val_df = pd.read_parquet(val_artifact_dir + '/' + VAL_ARTIFACT_NAME + '.parquet')
    eval_df = val_df.copy()

    # Load external data
    for art in [config.external_data_1, config.external_data_2, config.external_data_3, config.external_data_4, config.external_data_5]:
        if art != 'none':
            print(f'Loading external data {art}...')
            artifact = wandb.use_artifact(art)
            artifact_dir = artifact.download()
            ext_df = pd.read_parquet(artifact_dir + '/ext_data.parquet')
            train_df = pd.concat([train_df, ext_df], ignore_index=True)

    # Prepare references and labels from val set
    reference_df = get_reference_df(config.val_artifact)
#     all_labels = sorted(list(set(chain(*[x.tolist() for x in df.labels.values]))))
    all_labels = sorted(list(set(chain(*[x.tolist() for x in val_df.labels.values])))) #get from val df
    label2id = {l: i for i,l in enumerate(all_labels)}
    id2label = {v:k for k,v in label2id.items()}

    
    # Create the training and validation datasets
    tokenizer = AutoTokenizer.from_pretrained(config.training_model_path)
    train_ds = create_dataset(train_df, tokenizer, config.training_max_length, label2id)
    valid_ds = create_dataset(val_df, tokenizer, config.inference_max_length, label2id)
    
    #TEMP
    print(valid_ds.column_names)

    # Initialize the model and data collator
    model = AutoModelForTokenClassification.from_pretrained(
        config.training_model_path,
        num_labels=len(all_labels),
        id2label=id2label,
        label2id=label2id,
        ignore_mismatched_sizes=True
    )
    collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

    # Define the training arguments
    args = TrainingArguments(
        output_dir=config.output_dir, 
        fp16=config.fp16,
        learning_rate=config.learning_rate,
        num_train_epochs=config.num_train_epochs,
        per_device_train_batch_size=config.per_device_train_batch_size,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        report_to=config.report_to,
        evaluation_strategy=config.evaluation_strategy,
        do_eval=config.do_eval,
        save_total_limit=config.save_total_limit,
        logging_steps=config.logging_steps,
        lr_scheduler_type=config.lr_scheduler_type,
        warmup_ratio=config.warmup_ratio,
        weight_decay=config.weight_decay,
        
        #other args? metric_for_best_model, greater_is_better, 
    )

    #class weights based on dataset to go to CustomTrainer Class #TODO try without
    # Calculate class weights based on your dataset (TODO: move to config)
    class_weights = torch.tensor([1.]*12 + [config.o_weight]).to('cuda')

    #TODO try without custom trainer
    # Initialize Trainer with custom class weights
    trainer = CustomTrainer(
        model=model, 
        args=args, 
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        data_collator=collator, 
        tokenizer=tokenizer,
        compute_metrics=partial(compute_metrics, id2label=id2label, valid_ds=valid_ds, valid_df=reference_df, threshold=config.threshold),
        class_weights=class_weights,
    )
    
#     #TODO try this trainer
#     #inialize trainer for training and evaluation interface
#     trainer = Trainer(
#         model=model, 
#         args=args, 
#         train_dataset=train_ds,
#         eval_dataset= valid_ds,
#         data_collator=collator, 
#         tokenizer=tokenizer,
#         compute_metrics= partial(compute_metrics, id2label=id2label, valid_ds=valid_ds, valid_df=reference_df, threshold=config.threshold), #partial to fix all_label argument
#     )

    # Train the model
    trainer.train()

    print("trained")#DEBUG
    
    
    # Make predictions on the validation dataset
    preds = trainer.predict(valid_ds)
    
    print("predict done")#DEBUG

    # Compute the final metrics and log them to Weights & Biases
    print('Computing final metrics...')
    final_metrics = {
        f'final_f5_at_{threshold}': compute_metrics((preds.predictions, None), id2label, valid_ds, reference_df, threshold=threshold)['ents_f5']
        for threshold in [0.7,0.9,0.95]
    }
    wandb.log(final_metrics)
    print(final_metrics)

    # pick the best threshold from the final metrics and use it to generate preds_df
    best_threshold = float(max(final_metrics, key=final_metrics.get).split('_')[-1])
    wandb.config.best_threshold = best_threshold
    preds_df = parse_predictions(preds.predictions, id2label, valid_ds, threshold=best_threshold)

#     print('Visualizing errors...')
#     grouped_preds = preds_df.groupby('eval_row')[['document', 'token', 'label', 'token_str']].agg(list)
#     eval_df.set_index('index', inplace=True) # Set 'index' as the index of eval_df
#     # Merge on indices
#     viz_df = pd.merge(eval_df, grouped_preds, how='left', left_index=True, right_index=True)
#     viz_df = filter_errors(viz_df, preds_df)
#     viz_df['pred_viz'] = generate_htmls_concurrently(viz_df, tokenizer, preds.predictions, id2label, valid_ds, threshold=best_threshold)
#     nlp = spacy.blank("en")
#     htmls = viz_df.apply(lambda row: visualize(row, nlp), axis=1) # Use apply instead of iterrows
#     wandb_htmls = [wandb.Html(html) for html in htmls]
#     viz_df['gt_viz'] = wandb_htmls
#     viz_df.fillna("", inplace=True) # Fill NaN values in-place
#     viz_df = convert_for_upload(viz_df)
#     errors_table = wandb.Table(dataframe=viz_df)
#     wandb.log({'errors_table': errors_table})

    # Save the model and upload it to Kaggle
    os.makedirs(config.experiment, exist_ok=True)
    trainer.save_model(config.experiment)
    tokenizer.save_pretrained(config.experiment)
    # if training on a local machine, uncomment and fill in your username to upload the model to Kaggle
    # upload_kaggle_dataset(config.experiment, config.experiment, owner="thedrcat")
    print('Experiment finished, test it out on the inference notebook!')

# if __name__ == "__main__":
#     parse_args(config)
#     main(config)

#if in notebook then call to main is below in W and B section

In [10]:
# if helper function code is attached
# !cp /kaggle/input/pii-data-detection-training-code/* .

# W and B 

In [11]:
# make sure to attach key from secrets in add-ons
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")

import wandb
wandb.login(key=wandb_api_key)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
# runs training script

# Define the configuration for the experiment 
config.experiment = 'pii002'
config.raw_artifact='csci566sp24/pii/base_data:v2'
config.train_artifact = 'csci566sp24/pii/mini_no_overlap_data:v2'
config.val_artifact = 'csci566sp24/pii/val_data:v2'
config.learning_rate = 2e-5 
config.num_train_epochs =  2
config.training_model_path = "microsoft/deberta-v3-xsmall" 
config.per_device_train_batch_size = 8
config.per_device_eval_batch_size = 8
config.gradient_accumulation_steps = 4


main(config)

wandb: Currently logged in as: kasprisi (csci566sp24). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240410_043705-owmfmxft
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run flowing-dust-34
wandb: ⭐️ View project at https://wandb.ai/csci566sp24/pii
wandb: 🚀 View run at https://wandb.ai/csci566sp24/pii/runs/owmfmxft
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


...

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of DebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-xsmall and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an

Epoch,Training Loss,Validation Loss,Ents P,Ents R,Ents F5,Ents Per Type Id Num P,Ents Per Type Id Num R,Ents Per Type Id Num F5,Ents Per Type Street Address P,Ents Per Type Street Address R,Ents Per Type Street Address F5,Ents Per Type Phone Num P,Ents Per Type Phone Num R,Ents Per Type Phone Num F5,Ents Per Type Url Personal P,Ents Per Type Url Personal R,Ents Per Type Url Personal F5,Ents Per Type Username P,Ents Per Type Username R,Ents Per Type Username F5,Ents Per Type Name Student P,Ents Per Type Name Student R,Ents Per Type Name Student F5,Ents Per Type Email P,Ents Per Type Email R,Ents Per Type Email F5
0,1.362400,0.292306,0.000037,0.032836,0.000938,0.000008,0.666667,0.000211,0.000000,0.000000,0.000000,0.000261,0.176471,0.006553,0.004065,0.187500,0.068541,0.000000,0.000000,0.000000,0.000635,0.011070,0.006781,0.000000,0.000000,0.000000
1,0.226900,0.188853,0.002413,0.014925,0.012444,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.058824,0.057522,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.014760,0.014741,0.000000,0.000000,0.000000


trained


predict done
Computing final metrics...
{'final_f5_at_0.7': 0.0, 'final_f5_at_0.9': 0.012443763759931082, 'final_f5_at_0.95': 0.003488260661236224}
Experiment finished, test it out on the inference notebook!


# TODO
- move helper functions to seperate script